In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import tqdm
!jupyter nbextension enable --py widgetsnbextension


/home/antpc/.local/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/antpc/.local/lib/python3.8/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [39]:
class Discriminator(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.disc = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.BatchNorm1d(128),
            nn.LeakyReLU(0.01),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        return self.disc(x)
class Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super().__init__()
        
        self.gen = nn.Sequential(
            nn.Linear(z_dim, 256),
            nn.BatchNorm1d(256),
            nn.LeakyReLU(0.01),
            nn.Linear(256, img_dim),
            nn.Tanh(),  # normalize inputs to [-1, 1] so make outputs [-1, 1]
        )

    def forward(self, x):
        return self.gen(x)

In [45]:
class _Discriminator(nn.Module):
    def __init__(self, in_features):
        super(_Discriminator,self).__init__()
        self.l1=nn.Linear(in_features, 128)
        self.b=nn.BatchNorm1d(128)
        self.leak=nn.LeakyReLU(0.01)
        self.l2=nn.Linear(128, 1)
        self.sig=nn.Sigmoid()
    def forward(self, x):
        out=self.l1(x)
        # print(out.shape) # torch.Size([4, 128])
        out=self.b(out)
        # print(out.shape) # torch.Size([4, 128])
        out=self.leak(out)
        # print(out.shape) # torch.Size([4, 128])
        out=self.l2(out)
        # print(out.shape) # torch.Size([4, 1])
        out=self.sig(out)
        # print(out.shape) # torch.Size([4, 1])
        return out

class _Generator(nn.Module):
    def __init__(self, z_dim, img_dim):
        super(_Generator,self).__init__()
        self.l1=nn.Linear(z_dim, 256)
        self.ba=nn.BatchNorm1d(256)
        self.leak=nn.LeakyReLU(0.01)
        self.l2=nn.Linear(256, img_dim)
        self.tan=nn.Tanh() # normalize inputs to [-1, 1] so make outputs [-1, 1]

    def forward(self, x):
        # print(x.shape) # torch.Size([4, 100])
        out=self.l1(x)
        # print(out.shape) # torch.Size([4, 256])
        out=self.ba(out)
        # print(out.shape) # torch.Size([4, 256])
        out=self.leak(out)
        # print(out.shape) # torch.Size([4, 256])
        out=self.l2(out)
        # print(out.shape) # torch.Size([4, 784])
        out=self.tan(out)
        # print(out.shape) # torch.Size([2, 784])
        return out

In [54]:
b,z=4,100
image_dim = 28 * 28 * 1  # 784
disc=_Discriminator(image_dim)
gen = _Generator(z, image_dim)
real=torch.randn(b,784) 
# print(real.shape)# torch.Size([4, 784])
noise = torch.randn(b,z)
fake=gen(noise)
# print(fake.shape) # torch.Size([4, 784])
out= disc(fake)
# print(out.shape) # torch.Size([4, 1])
disc= disc(real)
# print(disc.shape) # torch.Size([4, 1])

torch.Size([4, 784])
torch.Size([4, 784])
torch.Size([4, 1])
torch.Size([4, 1])


In [31]:
# Hyperparameters etc.
device = "cuda:7" if torch.cuda.is_available() else "cpu"
lr = 3e-4
z_dim = 64
image_dim = 28 * 28 * 1  # 784
batch_size = 32
num_epochs = 100

disc = Discriminator(image_dim).to(device)
gen = Generator(z_dim, image_dim).to(device)
fixed_noise = torch.randn((batch_size, z_dim)).to(device)

In [5]:
transforms = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.5,), (0.5,)),
    ]
)

In [6]:
dataset = datasets.MNIST(root="dataset/", transform=transforms, download=True)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
opt_disc = optim.Adam(disc.parameters(), lr=lr)
opt_gen = optim.Adam(gen.parameters(), lr=lr)
gen_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt_gen, factor=0.1, patience=10, verbose=True
)
disc_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    opt_disc, factor=0.1, patience=10, verbose=True
)
criterion = nn.BCELoss()
# writer_fake = SummaryWriter(f"logs_lr_sched/fake")
# writer_real = SummaryWriter(f"logs_lr_sched/real")
step = 0

/mnt/disk1/conda/envs/gulshan/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


In [8]:
real,y=next(iter(loader))
real.shape


torch.Size([32, 1, 28, 28])

In [16]:
real = real.view(-1, 784).to(device)
batch_size = real.shape[0]
noise = torch.randn(batch_size, z_dim).to(device)
print(noise.shape) # torch.Size([32, 64])
### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
fake = gen(noise)
print(fake.shape) # torch.Size([32, 784])
disc_real = disc(real).view(-1)
print(disc_real.shape) # torch.Size([32])
lossD_real = criterion(disc_real, torch.ones_like(disc_real))
disc_fake = disc(fake)
print(disc_fake.shape) # torch.Size([32, 1])
disc_fake = disc(fake).view(-1)
print(disc_fake.shape) # torch.Size([32])
lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
lossD = (lossD_real + lossD_fake) / 2
disc.zero_grad()
lossD.backward(retain_graph=True)
opt_disc.step()

torch.Size([32, 64])
torch.Size([32, 784])
torch.Size([32])
torch.Size([32, 1])
torch.Size([32])


In [15]:
torch.ones_like(disc_real)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       device='cuda:7')

In [ ]:
# from tqdm.notebook import tqdm
# for epoch in tqdm(range(num_epochs),total=num_epochs):
#     for batch_idx, (real, _) in tqdm(enumerate(loader)):
#         real = real.view(-1, 784).to(device)
#         batch_size = real.shape[0]
#         noise = torch.randn(batch_size, z_dim).to(device)

#         ### Train Discriminator: max log(D(x)) + log(1 - D(G(z)))
#         fake = gen(noise)
#         disc_real = disc(real).view(-1)
#         lossD_real = criterion(disc_real, torch.ones_like(disc_real))
#         disc_fake = disc(fake).view(-1)
#         lossD_fake = criterion(disc_fake, torch.zeros_like(disc_fake))
#         lossD = (lossD_real + lossD_fake) / 2
#         disc.zero_grad()
#         lossD.backward(retain_graph=True)
#         opt_disc.step()
#         ### Train Generator: min log(1 - D(G(z))) <-> max log(D(G(z))
#         # where the second option of maximizing doesn't suffer from
#         # saturating gradients
#         output = disc(fake).view(-1)
#         lossG = criterion(output, torch.ones_like(output))
#         gen.zero_grad()
#         lossG.backward()
#         opt_gen.step()
#         if batch_idx == 0:
#             print(
#                 f"Epoch [{epoch}/{num_epochs}] Batch {batch_idx}/{len(loader)} \
#                       Loss D: {lossD:.4f}, loss G: {lossG:.4f}"
#             )

#             with torch.no_grad():
#                 fake = gen(fixed_noise).reshape(-1, 1, 28, 28)
#                 data = real.reshape(-1, 1, 28, 28)
#                 img_grid_fake = torchvision.utils.make_grid(fake, normalize=True)
#                 img_grid_real = torchvision.utils.make_grid(data, normalize=True)

#                 writer_fake.add_image(
#                     "Mnist Fake Images", img_grid_fake, global_step=step
#                 )
#                 writer_real.add_image(
#                     "Mnist Real Images", img_grid_real, global_step=step
#                 )
#                 step += 1

  0%|          | 0/100 [00:00<?, ?it/s]

0it [00:00, ?it/s]

Epoch [0/100] Batch 0/1875                       Loss D: 0.7191, loss G: 0.9167


0it [00:00, ?it/s]

Epoch [1/100] Batch 0/1875                       Loss D: 0.2075, loss G: 1.9385


...

Epoch [97/100] Batch 0/1875                       Loss D: 0.5538, loss G: 1.2274


0it [00:00, ?it/s]

Epoch [98/100] Batch 0/1875                       Loss D: 0.2596, loss G: 1.5137


0it [00:00, ?it/s]

Epoch [99/100] Batch 0/1875                       Loss D: 0.3965, loss G: 1.5539
